# Lección 1 - Análisis Inicial de Datos (IDA) y Generación de Dataset

El **Análisis Exploratorio de Datos (EDA)** es una fase fundamental para descubrir patrones y extraer valor de la información. Sin embargo, antes de explorar, debemos realizar un **Análisis Inicial de Datos (IDA)**, cuyo propósito es evaluar la calidad estructural del dataset, garantizando que la información sea confiable, coherente y esté lista para ser procesada estadísticamente.

### 🎯 Objetivo de la Lección
En este ejercicio práctico asumiremos el rol de la unidad de **Análisis Estratégico** de la empresa de ventas en línea **ComercioYA**. Utilizaremos las librerías de Python (`pandas` y `numpy`) para construir nuestra base de trabajo y auditar su calidad inicial mediante los siguientes pasos:

1.  **Generar un dataset comercial:** Construir una base de datos ficticia que simule el comportamiento histórico de nuestros clientes, incluyendo métricas clave como visitas, compras, montos totales, devoluciones y reseñas.
2.  **Clasificar variables:** Identificar y separar la información según su naturaleza estadística, distinguiendo claramente entre variables cuantitativas (numéricas) y categóricas (cualitativas).
3.  **Ejecutar el control de calidad (IDA):** Auditar el conjunto de datos para detectar inconsistencias, valores nulos (faltantes) y anomalías estructurales (como montos negativos) que deban ser tratadas antes de cualquier visualización o toma de decisiones.

In [7]:
import pandas as pd
import numpy as np

# 1. Configuración de la simulación
np.random.seed(42) # Fijamos semilla para mantener los mismos datos en cada ejecución
n_clientes = 1000

# 2. Generación de las variables solicitadas por ComercioYA
datos_simulados = {
    'id_cliente': range(1, n_clientes + 1),
    'visitas': np.random.randint(1, 150, n_clientes).astype(float), # Float para permitir valores nulos después
    'compras': np.random.randint(0, 30, n_clientes),
    'monto_total': np.random.uniform(5.0, 2500.0, n_clientes).round(2),
    'devoluciones': np.random.randint(0, 5, n_clientes),
    'reseña': np.random.choice(['Positiva', 'Neutral', 'Negativa'], n_clientes)
}

df_comercio = pd.DataFrame(datos_simulados)

# 3. Inyección de "datos sucios" para justificar el Análisis Inicial (IDA)
df_comercio.loc[15, 'monto_total'] = -350.0   # Inconsistencia lógica (monto negativo)
df_comercio.loc[128, 'visitas'] = np.nan      # Valor faltante (nulo)
df_comercio.loc[550, 'reseña'] = np.nan       # Valor faltante (nulo)
df_comercio.loc[888, 'monto_total'] = 18500.0 # Outlier extremo para analizar más adelante

# 4. Creación del archivo físico
nombre_archivo = 'comercio_ya_bruto.csv'
df_comercio.to_csv(nombre_archivo, index=False, encoding='utf-8')

print(f"✅ Archivo maestro '{nombre_archivo}' creado exitosamente.")
print(f"📊 Total de registros generados: {len(df_comercio)}")

✅ Archivo maestro 'comercio_ya_bruto.csv' creado exitosamente.
📊 Total de registros generados: 1000


### 1.Desarrollo: Generación y Exportación del Dataset

En esta primera etapa de ejecución, utilizaremos las librerías `pandas` y `numpy` para construir la base de datos histórica de **ComercioYA**. El siguiente bloque de código realiza cuatro acciones fundamentales:

1. **Simulación Controlada:** Generamos un volumen de 1,000 registros de clientes. Fijamos una semilla aleatoria (`seed=42`) para garantizar que la simulación sea reproducible, es decir, que los datos no cambien cada vez que ejecutemos el notebook.
2. **Creación de Variables:** Construimos exactamente las columnas solicitadas por la dirección estratégica: *visitas, compras, monto total, devoluciones* y *reseñas*.
3. **Inyección de "Datos Sucios" (Sabotaje controlado):** En el mundo real, los datos rara vez vienen perfectos. Para justificar nuestra etapa de Análisis Inicial (IDA) y simular un entorno verídico, introducimos intencionalmente:
   * **Inconsistencias lógicas:** Un cliente con un monto de compra negativo (`-350.0`).
   * **Valores faltantes (NaN):** Registros en blanco en las columnas de visitas y reseñas.
   * **Valores atípicos (Outliers):** Una compra con un monto desproporcionadamente alto (`$18,500.0`) que evaluaremos gráficamente más adelante.
4. **Persistencia de Datos:** Exportamos el DataFrame resultante a un archivo local llamado `comercio_ya_bruto.csv`. Este archivo actuará como nuestra fuente de datos "en bruto" para las fases posteriores de limpieza y visualización.

In [8]:
# 1. Cargamos el archivo que acabamos de crear
df_comercio_bruto = pd.read_csv('comercio_ya_bruto.csv')

# 2. Mostramos la información técnica y los tipos de datos (dtypes)
print("--- TIPOS DE VARIABLES DETECTADAS POR PANDAS ---")
print(df_comercio_bruto.dtypes)
print("\n")

# Extra: Usamos .info() que también nos da un adelanto de los valores no nulos
print("--- INFORMACIÓN GENERAL DEL DATASET ---")
df_comercio_bruto.info()

--- TIPOS DE VARIABLES DETECTADAS POR PANDAS ---
id_cliente        int64
visitas         float64
compras           int64
monto_total     float64
devoluciones      int64
reseña              str
dtype: object


--- INFORMACIÓN GENERAL DEL DATASET ---
<class 'pandas.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_cliente    1000 non-null   int64  
 1   visitas       999 non-null    float64
 2   compras       1000 non-null   int64  
 3   monto_total   1000 non-null   float64
 4   devoluciones  1000 non-null   int64  
 5   reseña        999 non-null    str    
dtypes: float64(2), int64(3), str(1)
memory usage: 47.0 KB


### 2.Clasificación de Variables para ComercioYA

Al ejecutar el análisis de tipos de datos (`dtypes`), confirmamos la siguiente estructura para nuestro diccionario de datos:

**1. Variables Cuantitativas:**
* `visitas` (float64): Cuantitativa discreta (está como flotante porque contiene valores nulos, lo cual Pandas maneja así por defecto).
* `compras` (int64): Cuantitativa discreta (número entero de transacciones).
* `monto_total` (float64): Cuantitativa continua (dinero, admite decimales).
* `devoluciones` (int64): Cuantitativa discreta.

**2. Variables Categóricas:**
* `reseña` (object): Categórica ordinal/nominal (texto que agrupa en 'Positiva', 'Neutral', 'Negativa').
* `id_cliente` (int64): Aunque Pandas lo detecta como número entero, comercialmente y estadísticamente es una **variable categórica nominal**, ya que es solo una etiqueta de identificación y no tiene sentido sumar o promediar los IDs de los clientes.

In [9]:
# 1. Carga de datos (aseguramos trabajar con el archivo físico)
df_ida = pd.read_csv('comercio_ya_bruto.csv')

print("--- AUDITORÍA DE VALORES NULOS ---")
print(df_ida.isnull().sum())
print("\n")

print("--- AUDITORÍA DE CONSISTENCIA NUMÉRICA ---")
# El describe() es la herramienta reina para detectar errores lógicos rápidamente
resumen = df_ida.describe()
print(resumen)

# Localizamos el registro con monto negativo para tener evidencia
monto_negativo = df_ida[df_ida['monto_total'] < 0]
print("\n--- REGISTROS CON MONTOS NEGATIVOS DETECTADOS ---")
print(monto_negativo)

--- AUDITORÍA DE VALORES NULOS ---
id_cliente      0
visitas         1
compras         0
monto_total     0
devoluciones    0
reseña          1
dtype: int64


--- AUDITORÍA DE CONSISTENCIA NUMÉRICA ---
        id_cliente     visitas      compras  monto_total  devoluciones
count  1000.000000  999.000000  1000.000000   1000.00000   1000.000000
mean    500.500000   76.404404    14.573000   1250.54706      1.985000
std     288.819436   43.636394     8.741343    901.37627      1.415902
min       1.000000    1.000000     0.000000   -350.00000      0.000000
25%     250.750000   38.500000     7.000000    593.07250      1.000000
50%     500.500000   76.000000    14.000000   1241.89500      2.000000
75%     750.250000  114.000000    22.000000   1857.83000      3.000000
max    1000.000000  149.000000    29.000000  18500.00000      4.000000

--- REGISTROS CON MONTOS NEGATIVOS DETECTADOS ---
    id_cliente  visitas  compras  monto_total  devoluciones   reseña
15          16      2.0        7       -

### 3: Detectar valores faltantes o inconsistencias (IDA)

El **Análisis Inicial de Datos (IDA)** es la auditoría técnica que realizamos para identificar problemas de calidad. No buscamos patrones de negocio todavía, sino errores en la "limpieza" o "integridad" de la tabla. 

En esta etapa para **ComercioYA**, nos enfocaremos en:
1.  **Integridad:** Cuántos datos nos faltan (valores nulos).
2.  **Validez Lógica:** Buscar valores que no tienen sentido en el mundo real (como montos negativos).
3.  **Distribución Preliminar:** Observar los rangos de las variables para detectar valores extremadamente altos (Outliers) que podrían distorsionar nuestros futuros promedios.

# 📝 Documentación de Hallazgos: Lección 1 (Análisis Inicial)

Tras la generación del dataset y la ejecución del **Análisis Inicial de Datos (IDA)**, se han identificado puntos críticos que condicionan la calidad de la información para los próximos pasos del proyecto de **ComercioYA**.

### 1. Clasificación y Diccionario de Variables
Se ha auditado la naturaleza de los datos para asegurar su correcto tratamiento estadístico:

| Variable | Tipo de Dato | Naturaleza | Descripción |
| :--- | :--- | :--- | :--- |
| `id_cliente` | int64 | Categórica | Identificador único del usuario (No operable matemáticamente). |
| `visitas` | float64 | Cuantitativa | Cantidad de ingresos al sitio web (Discreta). |
| `compras` | int64 | Cuantitativa | Volumen de transacciones realizadas (Discreta). |
| `monto_total` | float64 | Cuantitativa | Valor monetario de la transacción (Continua). |
| `devoluciones`| int64 | Cuantitativa | Cantidad de productos retornados (Discreta). |
| `reseña` | object | Categórica | Percepción cualitativa del cliente (Ordinal). |



### 2. Auditoría de Calidad (Hallazgos Críticos)

A partir de las funciones `.isnull().sum()` y `.describe()`, se detectaron las siguientes anomalías:

* **Valores Faltantes (Nulos):** Se identificaron vacíos en las columnas `visitas` y `reseña`. Estos registros representan una pérdida de trazabilidad que debe resolverse mediante limpieza o eliminación en la siguiente fase.
* **Inconsistencia Lógica:** Se detectó un valor de **-$350.0** en `monto_total`. Al ser una variable de ingresos, un valor negativo es un error de integridad del sistema que sesgaría cualquier cálculo de rentabilidad.
* **Presencia de Outliers:** El valor máximo registrado (\$18,500.0) se aleja drásticamente del comportamiento estándar del grupo, lo que sugiere la presencia de un cliente de alto valor o un error de digitación.

### 3. Conclusión de la Etapa
El dataset actual **no es apto** para el análisis descriptivo inmediato. Los hallazgos del IDA obligan a realizar una fase de **Data Cleaning** al inicio de la Lección 2 para garantizar que las decisiones comerciales se basen en evidencia real y no en distorsiones técnicas.